In [1]:
import sys
sys.path.append("..")
from main_libreview import *
from training_configs import *

In [2]:
# Dataset path 
DATASET_PATH = r"C:\Users\aralmeida\Downloads\LibreViewRawData"

tau = 1
kernel_size = 3
epochs = 1
batch_size = 4096
lr=0.001

epochs = 2
batch_size = 2
lr=0.001

In [3]:
test = testing 
dataset_path = DATASET_PATH

In [4]:
# launch_LibreView_experiments(testing, kernel_size, tau, lr, batch_size, epochs)

In [5]:
# Initial time
t0 = time.time()

# From the .csv files. extract the oldest year and store them to load them separately 
get_oldest_year_npys_from_LibreView_csv(dataset_path)

total_exps = len(test['sensor'])*len(test['N'])*len(test['step'])*len(test['PH'])*len(test['single_multi_step'])*len(test['partition'])*len(test['normalization'])*len(test['under_over_sampling'])*len(test['model'])*len(test['loss_function'])

# Iterate over the ID folders to generate the 4-folds 
for id in os.listdir(): 

    # Counter 
    i = 0

    # Consider only folders, not .npy or .txt files
    if ('npy' not in id) and ('txt' not in id): 
    
        # Get into the ID patient folder
        os.chdir(id)

        # Create dictionary to fill it with the results (one per patient) 
        results_dictionary = create_LibreView_results_dictionary()
        dict_test = {}

        # Avoid the generation of figures
        plt.ioff()
    
        # Only read the OLDEST year of recording
        recordings = np.load('oldest_1yr_CGM.npy')
        timestamps = np.load('oldest_1yr_CGM_timestamp.npy', allow_pickle=True)

        # For each patient (ID), the experiments included in "training_configs.py" are executed

        for sensors in range(len(test['sensor'])):
            for lengths in range(len(test['N'])):
                for steps in range(len(test['step'])):
                    for PHs in range(len(test['PH'])):
                        for predic_type in range(len(test['single_multi_step'])):
                            for partition in range(len(test['partition'])):
                                for norm_steps in range(len(test['normalization'])):
                                    for under_over_samp in range(len(test['under_over_sampling'])):
                                        for model_names in range(len(test['model'])):
                                            for loss_function in range(len(test['loss_function'])):
                                    
                                                # Update parameters 
                                                sensor = test['sensor'][sensors]
                                                N = test['N'][lengths]
                                                step = test['step'][steps]
                                                PH = test['PH'][PHs]
                                                single_multi_step = test['single_multi_step'][predic_type]
                                                data_partition = test['partition'][partition]
                                                normalization = test['normalization'][norm_steps]
                                                under_over_sampling = test['under_over_sampling'][under_over_samp]
                                                model_name = test['model'][model_names]
                                                loss_function = test['loss_function'][loss_function]

                                                # Create the key for current experiment 
                                                key = '{}_N{}_step{}_PH{}_{}_{}_{}_{}_{}'.format(single_multi_step, N, step, PH, data_partition, normalization,
                                                    under_over_sampling, model_name, loss_function)
                                                
                                                # Declare the dictionary key empty to fill it later if it has not been created yet 
                                                if key not in results_dictionary.keys():
                                                    results_dictionary[key] = {}
                                                else: 
                                                    pass

                                                if model_name not in results_dictionary[key].keys():
                                                    results_dictionary[key][model_name] = {}
                                                else:
                                                    pass

                                                # Store current working directory 
                                                cwd = os.getcwd()
                                                
                                                # If not created the directory correspondant with this configuration, create it
                                                subdirectory = r"\N{}\step{}\PH{}\{}\{}\norm_{}\{}_sampling\{}\{}".format(N, step, PH, single_multi_step,
                                                                                                                        data_partition, normalization, under_over_sampling, model_name, loss_function)
                                                if not os.path.exists(cwd+subdirectory):
                                                    os.makedirs(cwd+subdirectory)
                                            
                                                # Go that subdirectory 
                                                os.chdir(cwd+subdirectory)

                                                # Counter (with printing purposes)
                                                i = i+1

                                                print("~~~~~~~~~~~~~~~~~~~~~~~~~~\nRunning experiment %d/%d for Patient #%s :\n" % (i, total_exps, id))

                                                print("Configuration:\nsensor = %s\nN = %d\nstep = %d\nPH = %d\nsingle/multi step = %s\npartition = %s\nnorm = %s\nunder-over = %s\nmodel : %s\nloss funcion = %s\n" 
                                                % (sensor["NAME"], N, step, PH, single_multi_step, data_partition, normalization, under_over_sampling, model_name, loss_function))

                                                # Generate X and Y (only multistep supportd)
                                                if single_multi_step == 'multi':
                                                    X, Y, X_times, Y_times = get_LibreView_CGM_X_Y_multistep(recordings, timestamps, libreview_sensors, 
                                                            N, step, PH, plot = True, verbose = 0) 
                                                else:   
                                                    raise ValueError("Only 'multi' step prediction is supported in the LibreView-extracted data")
                                                
                                                
                                                # Data normalization
                                                if normalization == 'min-max':
                                                    X_norm = (X - np.min(X))/(np.max(X) - np.min(X))
                                                    Y_norm = (Y - np.min(X))/(np.max(X) - np.min(X))
                                                elif normalization == None: 
                                                    X_norm = X
                                                    Y_norm = Y 
                                                else: 
                                                    raise ValueError("Not valid normalization: only 'min-max' or None are currently supported")
                                            
                                                # Data partition
                                                # 1) Month-wise 4-folds partition
                                                if data_partition == 'month-wise-4-folds':                                                 
                                                    training_cv_folds  = month_wise_LibreView_4fold_cv(X_norm, Y_norm, X_times, Y_times, N)
                                                else: 
                                                    raise ValueError("Partition name not valid: currently 'month-wise-4-folds' is supported")
                                                
                                                
                                                # Apply (or not) undersampling or oversampling in training 
                                                if under_over_sampling == 'under':
                                                    X_train, Y_train  = undersample_normal_range_outputs(X, X_train, Y_train, multi_step=False, normalization = normalization, undersampling_factor = 2)
                                                elif under_over_sampling == None: 
                                                    pass

                                                if single_multi_step == 'multi':
                                                    if model_name == '1D-UNET':
                                                        model =  get_unet1D_multi_step(sensor, 
                                                                        N=N,
                                                                        input_features = 1,
                                                                        tau=tau,
                                                                        kernel_size=kernel_size, 
                                                                        PH=PH)
                                                        model.save_weights('initial_weights.h5')

                                                    elif model_name == '1D-UNET-non-compres':
                                                        model =  get_unet1D_nonCompres_multi_step(sensor, N=N,
                                                                        input_features = 1,
                                                                        tau=tau,
                                                                        kernel_size=kernel_size,
                                                                        PH=PH)
                                                        model.save_weights('initial_weights.h5')

                                                    elif model_name == 'DIL-1D-UNET':
                                                        model =  get_DIL_unet1D_multi_step(sensor, 
                                                                        N=N,
                                                                        input_features = 1,
                                                                        tau=tau,
                                                                        kernel_size=kernel_size,
                                                                        dilation_rate=1,
                                                                        PH=PH)
                                                        model.save_weights('initial_weights.h5')

                                                    elif model_name == 'LSTM':
                                                        model =  get_LSTM_multi_step(sensor, N=int(N),
                                                                        input_features = 1, PH=PH)
                                                        model.save_weights('initial_weights.h5')

                                                    elif model_name == '1D-UNET-LSTM':
                                                        model =  get_unet1DLSTM_multi_step(sensor, N=N,
                                                                        input_features = 1, PH=PH)
                                                        model.save_weights('initial_weights.h5')

                                                    elif model_name == 'StackedLSTM':
                                                        model = get_StackedLSTM_multi_step(sensor, N=int(N),
                                                                        input_features = 1, PH=PH)
                                                        model.save_weights('initial_weights.h5')
                                                    elif model_name == 'naive':
                                                        pass
                                                    else: 
                                                        raise ValueError("Model name not valid")
                                                else: 
                                                    raise ValueError("Only 'multi' step prediction is supported in the LibreView-extracted data")
                                                
                                                # Compute the number of predicted points that depends on the PH and the sensor sampling period
                                                predicted_points = round(PH/sensor['SAMPLE_PERIOD'])

                                                # If model is "naive", there is nothing to train
                                                if model_name != 'naive':
                                                    
                                                    if data_partition == 'month-wise-4-folds':
                                                        
                                                        # Train and evaluate each folds separately 
                                                        for fold in training_cv_folds.keys():

                                                            model.load_weights('initial_weights.h5')
                                                        
                                                            # If the directory fold is not created, create it
                                                            if fold not in os.listdir():
                                                                os.mkdir(fold)
                                                            
                                                            # Get into the fold directory
                                                            os.chdir(fold)
                                                            
                                                            # One model training per fold
                                                            train_model(sensor,
                                                                        model,
                                                                        X = training_cv_folds[fold]['X_train'],
                                                                        Y = training_cv_folds[fold]['Y_train'],
                                                                        N = N,
                                                                        predicted_points = predicted_points,
                                                                        epochs = epochs,
                                                                        batch_size = batch_size,
                                                                        lr = lr,
                                                                        fold = id+"-"+model_name+"-"+fold,
                                                                        loss_function = loss_function,
                                                                        verbose = 1 
                                                                        )
                                                        
                                                            # Model evaluation 
                                                            results_normal_eval = multi_step_model_evaluation(N, PH, id+"-"+model_name+"-"+fold, normalization, training_cv_folds[fold]['X_test'],
                                                                                                            training_cv_folds[fold]['Y_test'], predicted_points, X, loss_function)
                                                            
                                                            # # Refeed the model with the FIRST model output to evaluate the model. Aiming to reduce final error. 
                                                            # results_refeed_eval = multi_step_model_evaluation_refeed(N, PH, model_name, normalization, X_test, Y_test, X)
                                                            results_refeed_eval = []

                                                            results_dictionary[key][model_name][fold] = {'normal ': results_normal_eval, 'refeed': results_refeed_eval}

                                                            # Back to fold directory 
                                                            os.chdir('../..')

                                                    else:
                                                        raise ValueError("Partition name not valid: currently only 'month-wise-4-folds' training is supported")

                                                    # Back to id folder
                                                    os.chdir(cwd)

                                                    # Save updated dictionary 
                                                    with open('results_dictionary.json', 'w') as fp:
                                                        json.dump(results_dictionary, fp)                                                      

                                                    # Back to previous directory 
                                                    os.chdir('..')

                                                elif model_name == 'naive':
                                                    print("Naive model evaluation. Training step not needed. Only evaluation is performed")

                                                    # Do the naive prediction in all folds 
                                                    for fold in training_cv_folds.keys():
                                                        
                                                            # If the directory fold is not created, create it
                                                            if fold not in os.listdir():
                                                                os.mkdir(fold)
                                                            
                                                            # Get into the fold directory
                                                            os.chdir(fold)

                                                            if single_multi_step == 'multi':

                                                                # Non-refeed evaluation
                                                                results_normal_eval = multi_step_model_evaluation(N, PH, id+"-"+model_name+"-"+fold, normalization, training_cv_folds[fold]['X_test'],
                                                                                                            training_cv_folds[fold]['Y_test'], predicted_points, X, loss_function)
                                                                # os.chdir('..')

                                                                # # Refeed the model with the FIRST model output to evaluate the model. Aiming to reduce final error. 
                                                                # results_refeed_eval = multi_step_model_evaluation_refeed(N, PH, model_name, normalization, X_test, Y_test, X)
                                                                results_refeed_eval = []

                                                                results_dictionary[key][model_name][fold] = {'normal ': results_normal_eval, 'refeed': results_refeed_eval}
                                                                
                                                                # Back to previous directory 
                                                                os.chdir('../..')
                                                    
                                                            else: 
                                                                raise ValueError("Only 'multi' step prediction is supported in the LibreView-extracted data")
                                                            
                                                    # Back to id folder
                                                    os.chdir(cwd)

                                                    # Save updated dictionary 
                                                    with open('results_dictionary.json', 'w') as fp:
                                                        json.dump(results_dictionary, fp)                                                      

                                                    # Back to previous directory 
                                                    os.chdir('..')
                                                
                                                else: 
                                                    raise ValueError("Model name not valid")

    # Stop when the counter is equal to the total number of experiments
    if i == total_exps*29: # 29 is the number of patients
        break

# Final time
t1 = time.time()
print("Time elapsed: %d seconds" % (t1-t0))

Number of patients with at least one year of CGM data:  29
Dictionary loaded.

~~~~~~~~~~~~~~~~~~~~~~~~~~
Running experiment 1/4 for Patient #001 :

Configuration:
sensor = FreeStyle Libre X
N = 96
step = 1
PH = 30
single/multi step = multi
partition = month-wise-4-folds
norm = min-max
under-over = None
model : LSTM
loss funcion = root_mean_squared_error

Number of blocks of is 180

Discarded instances: 183
Running training using  GPU: ['device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:09:00.0, compute capability: 8.9']
Epoch 1/2
6479/6479 [==============================] - 35s 5ms/step - loss: 0.0030 - root_mean_squared_error: 0.0551 - ISO_adapted_loss: 0.0030
Epoch 2/2
6479/6479 [==============================] - 32s 5ms/step - loss: 0.0019 - root_mean_squared_error: 0.0436 - ISO_adapted_loss: 0.0019
dict_keys(['loss', 'root_mean_squared_error', 'ISO_adapted_loss'])

	End of the training. Model saved in C:\Users\aralmeida\Downloads\LibreViewRawData\1yr_npy_files\001\N96\

c:\Users\aralmeida\OneDrive - Universidad de Las Palmas de Gran Canaria\Doctorado\codigo\CGM_forecasting\tests\..\evaluation\multi_step\evaluation.py:705: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize = (20,10))


Running training using  GPU: ['device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:09:00.0, compute capability: 8.9']
Epoch 1/2
6460/6460 [==============================] - 41s 6ms/step - loss: 0.0030 - root_mean_squared_error: 0.0551 - ISO_adapted_loss: 0.0030
Epoch 2/2
3412/6460 [==============>...............] - ETA: 18s - loss: 0.0019 - root_mean_squared_error: 0.0437 - ISO_adapted_loss: 0.0019

: 

In [ ]:
model.reset_metrics()

In [ ]:
round(PH/sensor["SAMPLE_PERIOD"])

In [ ]:
model.